# LINCS Publications & Citations Report

Created by Brian M. Schilder, Moshe Silverstein, and Alexandra Keenan on 09/22/2018

In [1]:
import sys
print (sys.version)

3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 03:03:55) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


## Summary
- Plots:
	- Publications over time
	- Publications per LINCS center
	- Citations over time
	- Citations per center


- Repeat these plots for each of the following groupings:
	- Aggregated across internal LINCS centers

	- Divided amongst internal LINCS centers (label by center, not PI)
		1. Data Coordination and Integration Center (Mount Sinai, Avi Ma’ayan)
		2. Drug Toxicity Signature Generation Center (Mount Sinai, Ravi Iyengar)
		3. LINCS Proteomic Characterization Center for Signaling and Epigenetics (Broad Institute, Jacob Jaffe)
		4. Microenvironment Perturbagen (MEP) LINCS Center (Oregon Health & Science University, Joe Gray)
		5.	HMS LINCS Center (Peter Sorger)
		6.	LINCS Center for Transcriptomics (Broad Institute, Todd Golub)
		7.	NeuroLINCS Center  (UC-Irvine, Leslie Thompson)
	- Across all groups external to LINCS (E.g. L1000 paper, Canvas Browser). 


In [7]:
import pandas as pd
import seaborn as sn
pubs = pd.read_table('publications.tsv')

pubs.head()
pubs.columns


Index(['id', 'journal_name', 'show_at_home_order', 'abstract',
       'resource_links', 'pp_pages', 'issue', 'volume', 'pmc_id', 'pm_id',
       'doi', 'other_link', 'year_published', 'article_name',
       'assay_development', 'data_analysis', 'data_generation',
       'data_integration', 'data_standards', 'signature_generation',
       'software_development', 'review', 'created_at', 'updated_at',
       'analytical_method_development', 'center_pub'],
      dtype='object')

## Internal LINCS Publications: Aggregated

In [ ]:
years = pubs.dropna(subset=['year_published'])['year_published']
sn.distplot( years )